In [4]:
import pandas as pd
import smtplib
from pathlib import Path
import os

from email import encoders
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email.mime.image import MIMEImage

In [ ]:
def NewEnviarEmail(para, assunto, mensagem):
    try:
        msg = MIMEMultipart()

        msg['From'] = 'NotaFiscal@unicacontabil.com' 
        # msg['To'] = 'HenriqueCaziniFerreira@gmail.com'
        msg['To'] = para
        # msg['Cc'] = 'fiscal@unicacontabil.com'
        msg['Subject'] = assunto

        msg.attach(MIMEText(mensagem, 'plain'))
    
        #Assinatura Imagem
        jpg = "Assinatura Evelyn.jpg"
        fp = open(jpg, 'rb')
        msg_img = MIMEImage(fp.read())
        fp.close()
        msg_img.add_header('Content-ID', '<{}>'.format("Assinatura.jpg"))
        msg.attach(msg_img)

        #Enviar Email
        server = smtplib.SMTP('smtp.gmail.com: 587')
        server.starttls()
        # server.login('NotaFiscal@unicacontabil.com', keyring.get_password('.py', 'NotaFiscal'))
        server.login('NotaFiscal@unicacontabil.com', 'nfunica123')
        
        text = msg.as_string()
        server.sendmail(msg['From'], para.split(';'), text)
        server.quit()
    except:
        print("Erro!")

In [5]:
path_download = os.path.join(Path.home(), "Downloads")

# arq = 'C:/Users/Henrique/Downloads/ALÍQUOTAS MENSAIS.xlsx'
arq = os.path.join(path_download, 'ALÍQUOTAS MENSAIS.xlsx')

df = pd.read_excel(arq, header=2)
df2 = df.query("OBSERVAÇÕES != 'NÃO ENVIAR POR E-MAIL'").iloc[:, 0:29]
df2.set_index('CNPJ', inplace=True)

In [6]:
#Empresas Unicas
campos = "EMPRESA;EMAIL 01".split(';')
df3 = df2[campos].drop_duplicates(keep="first")

In [7]:
#Dicionarios Auxiliares
dMeses = {
    1:  ['Janeiro',     [1,3,4]],
    2:  ['Fevereiro',   [1,5,6]],
    3:  ['Março',       [1,7,8]],
    4:  ['Abril',       [1,9,10]],
    5:  ['Maio',        [1,11,12]],
    6:  ['Junho',       [1,13,14]],
    7:  ['Julho',       [1,15,16]],
    8:  ['Agosto',      [1,17,18]],
    9:  ['Setembro',    [1,19,20]],
    10: ['Outubro',     [1,21,22]],
    11: ['Novembro',    [1,23,24]],
    12: ['Dezembro',    [1,25,26]],
}

d = {
    'ANEXO 1' : ['Comércio', 'Alíquota ICMS: '],
    'ANEXO 2' : ['Industria', 'Alíquota ICMS: '],
    'ANEXO 3' : ['Serviço (III)', 'Alíquota ISS: '],
    'ANEXO 4' : ['Serviço (IV)', 'Alíquota ISS: '],
    'ANEXO 5' : ['Serviço (V)', 'Alíquota ISS: '],
    'FATOR R' : ['Serviço', 'Alíquota ISS: ']
}

In [8]:
# Main
for x,y in dMeses.items():
    print('ID: {} - Mes: {}' .format(x, y[0]))

mes = None
while mes is None:
    try:
        iMes = input('Digite o ID do mes selecionado: ')
        mes = dMeses[int(iMes)][0]
        cols = dMeses[int(iMes)][1]
    except:
        pass
print('{} / {}' .format(mes, cols) )
#Seleciona qual mes sera enviado o Email
df4 = df2.iloc[:,cols]

ID: 1 - Mes: Janeiro
ID: 2 - Mes: Fevereiro
ID: 3 - Mes: Março
ID: 4 - Mes: Abril
ID: 5 - Mes: Maio
ID: 6 - Mes: Junho
ID: 7 - Mes: Julho
ID: 8 - Mes: Agosto
ID: 9 - Mes: Setembro
ID: 10 - Mes: Outubro
ID: 11 - Mes: Novembro
ID: 12 - Mes: Dezembro
Junho / [1, 13, 14]


In [9]:
selecao = df3['EMAIL 01'].isnull()
# df3[selecao]

df3 = df3[~selecao]
df3["EMAIL 01"].replace(',' , ';' , regex=True, inplace=True)

In [10]:
# df4.info()
df3

,EMPRESA,EMAIL 01
CNPJ,,
2816376000102,ACF MARISTA LTDA,acfmarista@uol.com.br; rogeriorosacosta@gmail.com
14859991000177,BIZ CENTER LTDA,euripedes@bizcenter.com.br; comercial@bizcente...
26669556000169,BIOMEDIC LABORATORIO DE ANALISES CLINICAS LTDA,admlabnsaparecida@outlook.com; labbiomedic@hot...
37368198000170,AGENCIA FRANQUEADA 84 LTDA,agencia84@gmail.com
5120614000120,RADIOGYN DIAGNOSTICO ODONTOLOGICO POR IMAGEM LTDA,radiogyn@uol.com.br
...,...,...
10319905000182,PROJETUAL ARQUITETURA LTDA - ME,fernanda@projetualarquitetura.com.br
47057879000171,FLAVIA PEREIRA LIMA LTDA,limaflp@gmail.com
31073713000109,UNIO INSTITUTO DE SAUDE INTEGRADA LTDA,tratamentounio@gmail.com


In [14]:
ano = 2022
msgHead = """
Bom Dia,

Segue abaixo a alíquota do mês de {}/{}.
""" .format(mes, ano)

f1 = open("logAliquotas.txt", "w")  
for index, row1 in df3.iterrows(): 
    dfTemp = df4[(df4.index == index)]

    Title = 'ALÍQUOTA MENSAL ({}/{}) - ' .format(mes, ano) + row1["EMPRESA"] 
    EmailList = row1['EMAIL 01'] 

    msgBody = ""
    for index, row in dfTemp.iterrows(): 
        msgBody = msgBody + "\n"
        msgBody = msgBody + d[row['ANEXO SIMPLES']][0] + "\n"
        msgBody = msgBody + d[row['ANEXO SIMPLES']][1] + '{0:.2f}%'.format(row.iloc[1]*100) + "\n"
        msgBody = msgBody + 'Alíquota efetiva total: ' '{0:.2f}%'.format(row.iloc[2]*100) + "\n" 

    msgBody = msgBody.replace(".", ",")

    #Inicia o procedimento de envio de email
    msg = msgHead + msgBody
    # NewEnviarEmail(EmailList, Title, msg)
    f1.write(row1['EMPRESA'] + ' Enviado com Sucesso! \n')

    # Gravar .txt 
    path = "C:/Temp/Unica/"
    f = open(path + row1["EMPRESA"] +".txt", "x")
    f.write(Title + '\n')
    f.write(EmailList + '\n')
    f.write(msg + '\n')
    f.close()

f1.close()